# Preprocess data

In [26]:
import json

def load_and_convert_data(input_file):
    vehicles_data = {}

    with open(input_file, 'r') as f:
        for line in f:
            obj = json.loads(line.strip())
            vehicle_id = obj['vehicle_id']
            frame_ids = obj['frame_id']
            timestamps = [ts / 1000 for ts in obj['timestamp']]  # 毫秒转秒
            local_xs = [x * 0.3048 for x in obj['local_x']]  # 英尺转米
            local_ys = [y * 0.3048 for y in obj['local_y']]  # 英尺转米
            lane_ids = obj['lane_id']
            precedings = obj['preceding']
            followings = obj['following']
            v_length = obj['v_length'] * 0.3048

            vehicles_data[vehicle_id] = {
                'frame_id': frame_ids,
                'timestamp': timestamps,
                'local_x': local_xs,
                'local_y': local_ys,
                'lane_id': lane_ids,
                'preceding': precedings,
                'following': followings,
                'v_length': v_length
            }

    return vehicles_data

input_file = '../../trajectories-0400-0415.json'
vehicles_data = load_and_convert_data(input_file)
print("Load complete!")

Load complete!


# Find lane_change event

In [27]:
def detect_lane_changes(vehicles_data, output_file):
    lane_changes = []

    for vehicle_id, data in vehicles_data.items():
        lane_id = data['lane_id']
        timestamps = data['timestamp']
        preceding = data['preceding']
        following = data['following']
        v_length = data['v_length']

        for i in range(1, len(lane_id)):
            if lane_id[i] != lane_id[i - 1]:
                lane_changes.append({
                    'vehicle_id': vehicle_id,
                    'v_length': v_length,
                    'before_lane_id': lane_id[i-1],
                    'after_lane_id': lane_id[i],
                    'before_change_timestamp': timestamps[i - 1],
                    'after_change_timestamp': timestamps[i],
                    'before_preceding_id': preceding[i-1],
                    'before_following_id': following[i-1],
                    'after_preceding_id': preceding[i],
                    'after_following_id': following[i]
                })

    with open(output_file, 'w') as f:
        for change in lane_changes:
            json.dump(change, f)
            f.write('\n')

output_file = 'trajectories-0400-0415-lane-change-for-IDM.json'
detect_lane_changes(vehicles_data, output_file)

print('Record complete!')

Record complete!


# Step function

In [28]:
def data_for_gap_before_change(start_time, end_time, vehicles_data, vehicle_id, position_list):
    current_data = vehicles_data[vehicle_id]
    current_timestamps = current_data['timestamp']
    current_local_y = current_data['local_y']
    for i, ts in enumerate(current_timestamps):
        if start_time <= ts <= end_time:
            position_list.append(current_local_y[i])
    
def data_for_gap_after_change(start_time, end_time, vehicles_data, vehicle_id, position_list):
    current_data = vehicles_data[vehicle_id]
    current_timestamps = current_data['timestamp']
    current_local_y = current_data['local_y']
    for i, ts in enumerate(current_timestamps):
        if start_time <= ts <= end_time:
            position_list.append(current_local_y[i])

def calculation_for_gap(vel_length, position_list_pre, position_list_fol, result_list):
    calculation_list = [ pre - fol - vel_length for pre, fol in zip(position_list_pre, position_list_fol)]
    result_list.extend(calculation_list)
    
def merge_position(position_before_list, position_after_list, result_list):
    result_list.extend(position_before_list + position_after_list)
    
def calculation_for_vel(position_list, result_list, delta_t = 0.1):
    i = 0
    while i < len(position_list)-1:
        velocity = (position_list[i + 1] - position_list[i]) / delta_t
        result_list.append(velocity)
        i = i + 1
    while len(result_list) < len(position_list):
        result_list.append(result_list[-1])
        
def calculation_for_acc(velocity_list, result_list, delta_t = 0.1):
    i = 0
    while i < len(velocity_list)-1:
        acceleration = (velocity_list[i + 1] - velocity_list[i]) / delta_t
        result_list.append(velocity)
        i = i + 1
    while len(result_list) < len(velocity_list):
        result_list.append(result_list[-1])

# Complete the IDM-model file

In [29]:
def create_idm_modeling_file(vehicles_data, lane_changes_file, idm_output_file, delta_t=40):
    idm_data = []

    with open(lane_changes_file, 'r') as f:
        for line in f:
            change = json.loads(line.strip())
            vehicle_id = change['vehicle_id']
            v_length = change['v_length']
            before_change = change['before_change_timestamp']
            after_change = change['after_change_timestamp']
            
            vehicle_data = vehicles_data[vehicle_id]
            timestamps = vehicle_data['timestamp']
            local_y = vehicle_data['local_y']
            
            start_time = before_change - delta_t / 2
            end_time = after_change + delta_t / 2

            # 检查目标车辆的时间窗口内数据是否完整
            if start_time not in timestamps or end_time not in timestamps:
                continue

            before_preceding_id = change['before_preceding_id']
            after_preceding_id = change['after_preceding_id']
            before_following_id = change['before_following_id']
            after_following_id = change['after_following_id']
            
            # 变道前后前后车不存在的话直接忽略
            if not before_preceding_id or not after_preceding_id or not after_following_id or not before_following_id:
                continue

            # 检查变道前前车的时间窗口内数据是否完整
            if before_preceding_id and before_preceding_id in vehicles_data:
                preceding_data = vehicles_data[before_preceding_id]
                preceding_timestamps = preceding_data['timestamp']
                v_before_length = preceding_data['v_length']
                if start_time not in preceding_timestamps or end_time not in preceding_timestamps:
                    continue
                    
            # 检查变道前后车的时间窗口内数据是否完整
            if before_following_id and before_following_id in vehicles_data:
                following_data = vehicles_data[after_following_id]
                following_timestamps = following_data['timestamp']
                if start_time not in following_timestamps or end_time not in following_timestamps:
                    continue

            # 检查变道后前车的时间窗口内数据是否完整
            if after_preceding_id and after_preceding_id in vehicles_data:
                preceding_data = vehicles_data[after_preceding_id]
                preceding_timestamps = preceding_data['timestamp']
                v_after_length = preceding_data['v_length']
                if start_time not in preceding_timestamps or end_time not in preceding_timestamps:
                    continue

            # 检查变道后后车的时间窗口内数据是否完整
            if after_following_id and after_following_id in vehicles_data:
                following_data = vehicles_data[after_following_id]
                following_timestamps = following_data['timestamp']
                if start_time not in following_timestamps or end_time not in following_timestamps:
                    continue

            idm_entry = {
                'vehicle_id': vehicle_id,
                'before_preceding_id': before_preceding_id,
                'before_following_id': before_following_id,
                'after_preceding_id': after_preceding_id,
                'after_following_id': after_following_id,
                'before_lane_id': change['before_lane_id'],
                'after_lane_id': change['after_lane_id'],
                'v_before_preceding_length': v_before_length,
                'v_after_preceding_length': v_after_length,
                'v_length': v_length,
                'timestamp': [],
                'before_change_timestamp': before_change,
                'after_change_timestamp': after_change,
                'focus_position': [],
                'focus_velocity': [],
                'before_following_init_velocity': None,
                'before_following_inti_position': None,
                'after_following_init_velocity': None,
                'after_following_init_position': None,
                'before_preceding_after_change_position': [],
                'before_preceding_after_change_velocity': [],
                'after_preceding_before_change_velocity': [],
                'after_preceding_before_change_position': []
            }
            
            # 记录变道前后车和变道后后车的初始信息
            current_data = vehicles_data[before_following_id]
            current_timestamp = current_data['timestamp']
            current_position = current_data['local_y']
            for i, ts in enumerate(current_timestamp):
                if ts == start_time:
                    idm_entry['before_following_init_position'] = current_position[i]
                    idm_entry['before_following_init_velocity'] = (current_position[i+1] - current_position[i]) / 0.1 * 2.237
                    break
                    
            current_data = vehicles_data[after_following_id]
            current_timestamp = current_data['timestamp']
            current_position = current_data['local_y']
            for i, ts in enumerate(current_timestamp):
                if ts == start_time:
                    idm_entry['after_following_init_position'] = current_position[i]
                    idm_entry['after_following_init_velocity'] = (current_position[i+1] - current_position[i]) / 0.1 * 2.237
                    break
        
            # 记录时间帧和变道车辆位置
            for i, ts in enumerate(timestamps):
                if start_time <= ts <= end_time:
                    idm_entry['timestamp'].append(ts)
                    idm_entry['focus_position'].append(local_y[i])
                    
            # 记录变道前前车和变道后前车的相应位置
            current_data = vehicles_data[before_preceding_id]
            current_timestamp = current_data['timestamp']
            current_position = current_data['local_y']
            for i, ts in enumerate(current_timestamp):
                if after_change <= ts <= end_time:
                    idm_entry['before_preceding_after_change_position'].append(current_position[i])
            
            current_data = vehicles_data[after_preceding_id]
            current_timestamp = current_data['timestamp']
            current_position = current_data['local_y']
            for i, ts in enumerate(current_timestamp):
                if start_time <= ts <= before_change:
                    idm_entry['after_preceding_before_change_position'].append(current_position[i])
            
            # 计算并记录变道车辆速度,变道前前车和变道后前车的相应速度
            calculation_for_vel(idm_entry['focus_position'], idm_entry['focus_velocity'], delta_t = 0.1)
            calculation_for_vel(idm_entry['before_preceding_after_change_position'], idm_entry['before_preceding_after_change_velocity'], delta_t = 0.1)            
            calculation_for_vel(idm_entry['after_preceding_before_change_position'], idm_entry['after_preceding_before_change_velocity'], delta_t = 0.1)

            idm_data.append(idm_entry)

            if len(idm_data) % 400 == 0:
                print(f"Processed {len(idm_data)} IDM entries")

    with open(idm_output_file, 'w') as f:
        for entry in idm_data:
            json.dump(entry, f)
            f.write('\n')

    print(f"Total IDM entries: {len(idm_data)}")

create_idm_modeling_file(vehicles_data, 'trajectories-0400-0415-lane-change-for-IDM.json', 'trajectories-0400-0415-idm_modeling_data.json', delta_t=40)

print('IDM modeling data complete!')

Total IDM entries: 181
IDM modeling data complete!


# Complete the empirical file

In [30]:
def create_real_values_file(vehicles_data, lane_changes_file, real_output_file, delta_t=40):
    real_data = []

    with open(lane_changes_file, 'r') as f:
        for line in f:
            change = json.loads(line.strip())
            vehicle_id = change['vehicle_id']
            v_length = change['v_length']
            before_change = change['before_change_timestamp']
            after_change = change['after_change_timestamp']
            
            vehicle_data = vehicles_data[vehicle_id]
            timestamps = vehicle_data['timestamp']
            local_y = vehicle_data['local_y']
            
            start_time = before_change - delta_t / 2
            end_time = after_change + delta_t / 2

            # 检查目标车辆的时间窗口内数据是否完整
            if start_time not in timestamps or end_time not in timestamps:
                continue

            before_preceding_id = change['before_preceding_id']
            after_preceding_id = change['after_preceding_id']
            before_following_id = change['before_following_id']
            after_following_id = change['after_following_id']
            
            # 变道前后前后车不存在的话直接忽略
            if not before_preceding_id or not after_preceding_id or not after_following_id or not before_following_id:
                continue

            # 检查变道前前车的时间窗口内数据是否完整
            if before_preceding_id and before_preceding_id in vehicles_data:
                preceding_data = vehicles_data[before_preceding_id]
                preceding_timestamps = preceding_data['timestamp']
                v_before_length = preceding_data['v_length']
                if start_time not in preceding_timestamps or end_time not in preceding_timestamps:
                    continue
                    
            # 检查变道前后车的时间窗口内数据是否完整
            if before_following_id and before_following_id in vehicles_data:
                following_data = vehicles_data[after_following_id]
                following_timestamps = following_data['timestamp']
                if start_time not in following_timestamps or end_time not in following_timestamps:
                    continue

            # 检查变道后前车的时间窗口内数据是否完整
            if after_preceding_id and after_preceding_id in vehicles_data:
                preceding_data = vehicles_data[after_preceding_id]
                preceding_timestamps = preceding_data['timestamp']
                v_after_length = preceding_data['v_length']
                if start_time not in preceding_timestamps or end_time not in preceding_timestamps:
                    continue

            # 检查变道后后车的时间窗口内数据是否完整
            if after_following_id and after_following_id in vehicles_data:
                following_data = vehicles_data[after_following_id]
                following_timestamps = following_data['timestamp']
                if start_time not in following_timestamps or end_time not in following_timestamps:
                    continue

            real_entry = {
                'vehicle_id': vehicle_id,
                'before_lane_id': change['before_lane_id'],
                'after_lane_id': change['after_lane_id'],
                'before_following_id': before_following_id,
                'after_following_id': after_following_id,
                'timestamp': [],
                'before_following_before_gap': [],
                'before_following_after_gap': [],
                'after_following_before_gap': [],
                'after_following_after_gap': [],
                'before_following_velocities': [],
                'before_following_accelerations': [],
                'after_following_velocities': [],
                'after_following_accelerations': []
            }
            
            # 记录时间帧
            for i, ts in enumerate(timestamps):
                if start_time <= ts <= end_time:
                    real_entry['timestamp'].append(ts)
        
            # 记算gap
            # 准备计算gap的数据
            focus_vehicle_before_change_position = []
            focus_vehicel_after_change_position = []
            before_preceding_before_change_position = []
            before_preceding_after_change_position = []
            before_following_before_change_position = []
            before_following_after_change_position = []
            after_preceding_before_change_position = []
            after_preceding_after_change_position =[]
            after_following_before_change_position = []
            after_following_after_change_position =[]
            # 目标
            data_for_gap_before_change(start_time, before_change, vehicles_data, vehicle_id, focus_vehicle_before_change_position)
            data_for_gap_after_change(after_change, end_time, vehicles_data, vehicle_id, focus_vehicel_after_change_position)
            # 前前        
            data_for_gap_before_change(start_time, before_change, vehicles_data, before_preceding_id, before_preceding_before_change_position)
            data_for_gap_after_change(after_change, end_time, vehicles_data, before_preceding_id, before_preceding_after_change_position)
            # 前后
            data_for_gap_before_change(start_time, before_change, vehicles_data, before_following_id, before_following_before_change_position)
            data_for_gap_after_change(after_change, end_time, vehicles_data, before_following_id, before_following_after_change_position)
            # 后前
            data_for_gap_before_change(start_time, before_change, vehicles_data, after_preceding_id, after_preceding_before_change_position)
            data_for_gap_after_change(after_change, end_time, vehicles_data, after_preceding_id, after_preceding_after_change_position)
            # 后后
            data_for_gap_before_change(start_time, end_time, vehicles_data, after_following_id, after_following_before_change_position)
            data_for_gap_after_change(after_change, end_time, vehicles_data, after_following_id, after_following_after_change_position)
            # 计算变道前后车的两个gap
            calculation_for_gap(v_length, focus_vehicle_before_change_position, before_following_before_change_position, real_entry['before_following_before_gap'])
            calculation_for_gap(v_before_length, before_preceding_after_change_position, before_following_after_change_position, real_entry['before_following_after_gap'])
            # 计算变道后后车的两个gap
            calculation_for_gap(v_after_length, after_preceding_before_change_position, after_following_before_change_position, real_entry['after_following_before_gap'])
            calculation_for_gap(v_length, focus_vehicel_after_change_position, after_following_after_change_position, real_entry['after_following_after_gap'])
            
            # 计算变道前后车的真实速度与加速度
            # 准备两个后车的位置数据
            before_following_position = []
            after_following_position = []
            merge_position(before_following_before_change_position, before_following_after_change_position, before_following_position)
            merge_position(after_following_before_change_position, after_following_after_change_position, after_following_position)
            # 计算变道前后车的位置与加速度
            calculation_for_vel(before_following_position, real_entry['before_following_velocities'], delta_t = 0.1)
            calculation_for_vel(real_entry['before_following_velocities'], real_entry['before_following_accelerations'], delta_t = 0.1)
            # 计算变道后后车的位置与加速度
            calculation_for_vel(before_following_position, real_entry['after_following_velocities'], delta_t = 0.1)
            calculation_for_vel(real_entry['after_following_velocities'], real_entry['after_following_accelerations'], delta_t = 0.1)
            
            real_entry['before_following_velocities'][:] = [x * 2.237 for x in real_entry['before_following_velocities']]
            real_entry['after_following_velocities'][:] = [x * 2.237 for x in real_entry['after_following_velocities']]

            real_data.append(real_entry)

            if len(real_data) % 400 == 0:
                print(f"Processed {len(real_data)} real value entries")

    with open(real_output_file, 'w') as f:
        for entry in real_data:
            json.dump(entry, f)
            f.write('\n')

    print(f"Total real value entries: {len(real_data)}")

create_real_values_file(vehicles_data, 'trajectories-0400-0415-lane-change-for-IDM.json', 'trajectories-0400-0415-real_values_data.json', delta_t=40)
  
print('Real values data complete!')

Total real value entries: 181
Real values data complete!
